# 영화 평점 게시글을 이용한 정서 분석

## 목차
- 가설
- 연구방법
- 결론

## 1. 가설
- 나이브 베이즈 분류기와 SVM 분류기를 통해 영화 평점 게시글의 정서를 파악할 수 있다.

## 2. 연구방법
- 2.1 데이터 수집
- 2.2 분류기 생성
- 2.3 분류기 평가

### 2.1 데이터 수집
- Movie Review Data 이용
- Sentiment polarity(positive or negative)를 분류함
- 5331 positive and 5331 negative processed sentences / snippets. Introduced in Pang/Lee ACL 2005. Released July 2005.
- 출처: http://www.cs.cornell.edu/People/pabo/movie-review-data/

### 2.2 분류기 생성

#### 2.2.1 Load data
- Positive post 5331개와 negative post 5331개를 로드함 (rt-polarity.pos, rt-polarity.neg)
- 최소 2글자 이상
- 소문자로 변환

In [2]:
from numpy import *

def textParse(bigString):    #input is big string, #output is word list
    listOfTokens = bigString.strip().split(' ')
    return [tok.lower() for tok in listOfTokens if len(tok) > 2] 

def loadDataSet():
    # positive
    fr=open('data/rt-polarity.pos')
    posPosting = [textParse(inst) for inst in fr.readlines()]
    posLen = len(posPosting)
    posLabels = ones(posLen).tolist()
    
    # negative
    fr=open('data/rt-polarity.neg')
    negPosting = [textParse(inst) for inst in fr.readlines()]
    negLen = len(negPosting)
    negLabels = zeros(negLen).tolist()
    
    # merge
    posPosting.extend(negPosting)
    posLabels.extend(negLabels)
   
    return posPosting, posLabels

In [3]:
listOPosts,listClasses = loadDataSet()
print 'length of post list: %d' % len(listOPosts)
print 'first post of post list: %s' % listOPosts[0]
print 'first class of class list : %s' % listClasses[0]

length of post list: 10662
first post of post list: ['the', 'rock', 'destined', 'the', '21st', "century's", 'new', 'conan', 'and', 'that', "he's", 'going', 'make', 'splash', 'even', 'greater', 'than', 'arnold', 'schwarzenegger', 'jean-claud', 'van', 'damme', 'steven', 'segal']
first class of class list : 1.0


#### 2.2.2 Naive basis code 
- 분류기 생성

In [4]:
def createVocabList(dataSet):
    vocabSet = set([])  #create empty set
    for document in dataSet:
        vocabSet = vocabSet | set(document) #union of the two sets
    return list(vocabSet)

def setOfWords2Vec(vocabList, inputSet):
    returnVec = [0]*len(vocabList)
    for word in inputSet:
        if word in vocabList:
            returnVec[vocabList.index(word)] = 1
        #else: print "the word: %s is not in my Vocabulary!" % word
    return returnVec

def trainNB0(trainMatrix,trainCategory):
    numTrainDocs = len(trainMatrix)
    numWords = len(trainMatrix[0])
    pAbusive = sum(trainCategory)/float(numTrainDocs)
    p0Num = ones(numWords); p1Num = ones(numWords)      #change to ones() 
    p0Denom = 2.0; p1Denom = 2.0                        #change to 2.0
    for i in range(numTrainDocs):
        if trainCategory[i] == 1:
            p1Num += trainMatrix[i]
            p1Denom += sum(trainMatrix[i])
        else:
            p0Num += trainMatrix[i]
            p0Denom += sum(trainMatrix[i])
    p1Vect = log(p1Num/p1Denom)          #change to log()
    p0Vect = log(p0Num/p0Denom)          #change to log()
    return p0Vect,p1Vect,pAbusive

In [13]:
import time

myVocabList = createVocabList(listOPosts)
trainMat=[]
# training
for postinDoc in listOPosts:
    trainMat.append(setOfWords2Vec(myVocabList, postinDoc))
    
start_time = time.clock() # start time

p0V,p1V,pAb = trainNB0(array(trainMat),array(listClasses)) # training naive basis

end_time = time.clock() # end time
process_time = end_time - start_time # process time

print p0V
print p1V
print pAb
print "process time is: %s" % process_time

[-10.13191048  -8.34015101 -10.53737558 ..., -11.23052277 -10.53737558
  -9.8442284 ]
[ -9.85111473  -8.52935889 -10.54426191 ..., -10.1387968   -9.85111473
 -11.23740909]
0.5
process time is: 10.2923255532


#### 2.2.3 Support vector machine code
- 분류기 생성

In [6]:
import svmutil as svm
def trainSVM(trainMatrix, trainCategory):
    svm.svm_model.predict = lambda self, x: svm.svm_predict([0], [x], self)[0][0]

    prob = svm.svm_problem(trainCategory, trainMatrix)
    param = svm.svm_parameter()
    param.kernel_type = svm.LINEAR
    param.C = 10
    
    model = svm.svm_train(prob, param)
    return model

In [14]:
listClasses = map(int, listClasses)
start_time = time.clock() # start time

model = trainSVM(trainMat, listClasses) # training svm

end_time = time.clock() # end time
process_time = end_time - start_time # process time
print "process time is: %s" % process_time

process time is: 82.7104038715


### 2.3 분류기 평가

#### 2.3.1 Naive basis

In [8]:
def classifyNB(vec2Classify, p0Vec, p1Vec, pClass1):
    p1 = sum(vec2Classify * p1Vec) + log(pClass1)    #element-wise mult
    p0 = sum(vec2Classify * p0Vec) + log(1.0 - pClass1)
    if p1 > p0:
        return 1
    else: 
        return 0
    
def testingNB():
    listOPosts,listClasses = loadDataSet()
    myVocabList = createVocabList(listOPosts)
    trainMat=[]
    # training
    for postinDoc in listOPosts:
        trainMat.append(setOfWords2Vec(myVocabList, postinDoc))
    p0V,p1V,pAb = trainNB0(array(trainMat),array(listClasses))
    print p0V, p1V, pAb
    # test
    errorCount = 0
    for testIndex in range(0, len(listOPosts)):
        thisDoc = array(setOfWords2Vec(myVocabList, listOPosts[testIndex]))    
        if classifyNB(thisDoc,p0V,p1V,pAb) != listClasses[testIndex]:
            errorCount += 1
    print 'the error rate is:', float(errorCount)/len(listOPosts)*100

In [15]:
start_time = time.clock() # start time

testingNB() # test naive basis

end_time = time.clock() # end time
process_time = end_time - start_time # process time
print "process time is: %s" % process_time

[-10.13191048  -8.34015101 -10.53737558 ..., -11.23052277 -10.53737558
  -9.8442284 ] [ -9.85111473  -8.52935889 -10.54426191 ..., -10.1387968   -9.85111473
 -11.23740909] 0.5
the error rate is: 6.3215156631
process time is: 144.164912856


#### 2.3.2 Support vector machine

In [16]:
start_time = time.clock() # start time

svm.svm_predict(listClasses, trainMat, model)

end_time = time.clock() # end time
process_time = end_time - start_time # process time
print "process time is: %s" % process_time

Accuracy = 99.9437% (10656/10662) (classification)
process time is: 38.6482202593


### 3. 결론

#### Accuracy
- 나이브 베이즈 분류기의 오차율은 약 6%로 확인됨
- SVM 분류기의 오차율은 약 0.06%로 확인됨

#### Process time
- 나이브 베이즈 분류기의 훈련 시간은 10.29초로 확인됨
- SVM 분류기의 훈련 시간은 82.71초로 확인됨
- 나이브 베이즈 분류기의 분류 시간은 144.16초로 확인됨
- SVM 분류기의 분류 시간은 38.64초로 확인됨

#### Result
- SVM 분류기의 성능이 나이브 베이즈 분류기의 성능보다 뛰어난 것으로 판별됨
- 나이브 베이즈 분류기의 훈련 시간이 SVM 분류기의 훈련 시간보다 빠른 것으로 판별됨
- SVM 분류기의 분류 시간이 나이브 베이즈 분류기의 분류 시간보다 빠른 것으로 판별됨

#### Discussion
- SVM 분류기가 나이브 베이즈 분류기보다 훈련 시간은 더 오래 걸리지만, 분류 시간과 정확도는 더 뛰어난 것으로 밝혀짐
- 실제 정서 분석을 위해서는 SVM 분류기가 더 적합하다고 판단됨